<a href="https://colab.research.google.com/github/Amirnaderiy/GAN/blob/main/Synthetic_Ultrasound.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import os
import requests
from google.colab import drive

# Mount your Google Drive
drive.mount('/content/drive')

# Set the path to the models directory on your Google Drive
models_dir = '/content/drive/MyDrive/stylegan2/models'

# Create the models directory if it doesn't exist
os.makedirs(models_dir, exist_ok=True)

# Set the URL for the pre-trained weights
url = 'https://nvlabs-fi-cdn.nvidia.com/stylegan2/networks/stylegan2-ffhq-config-f.pkl'

# Download the pre-trained weights
response = requests.get(url)
with open(os.path.join(models_dir, 'ffhq.pkl'), 'wb') as f:
    f.write(response.content)

print('Download complete!')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Download complete!


In [9]:
import tensorflow as tf
from google.colab import drive
import numpy as np
import cv2
import os
from tensorflow.python.framework.ops import disable_eager_execution

# Disable eager execution
disable_eager_execution()


In [15]:
!rm -rf stylegan2

!git clone https://github.com/NVlabs/stylegan2.git
!python stylegan2/download_weights.py


Cloning into 'stylegan2'...
remote: Enumerating objects: 138, done.
remote: Total 138 (delta 0), reused 0 (delta 0), pack-reused 138
Receiving objects: 100% (138/138), 594.91 KiB | 8.75 MiB/s, done.
Resolving deltas: 100% (64/64), done.
python3: can't open file '/content/stylegan2/download_weights.py': [Errno 2] No such file or directory


In [18]:
np.random.seed(42)
tf.random.set_seed(42)

# Load your own image
drive.mount('/content/drive', force_remount=True)
image_folder = '/content/drive/MyDrive/B-Mode Ultrasound/'
image_files = os.listdir(image_folder)

for image_file in image_files:
    # Construct the full path to the image
    image_path = os.path.join(image_folder, image_file)

    # Load the image
    original_image = cv2.imread(image_path)

    # Preprocess the image
    preprocessed_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
    preprocessed_image = cv2.resize(preprocessed_image, (512, 512))  # Resize to match StyleGAN2 input size
    preprocessed_image = (preprocessed_image / 255.0) * 2 - 1  # Normalize pixel values to [-1, 1]


    # Reshape the image for StyleGAN2 input
    image_batch = np.expand_dims(preprocessed_image, axis=0)

    # Load StyleGAN2 model
    if 'sess' in globals(): sess.close()
    sess = tf.compat.v1.InteractiveSession()
    saver = tf.compat.v1.train.import_meta_graph('/content/drive/MyDrive/stylegan2/models/ffhq.pkl.meta')
    saver.restore(sess, '/content/drive/MyDrive/stylegan2/models/ffhq.pkl')


    # Access the generator's input and output tensors
    graph = tf.compat.v1.get_default_graph()
    latent_vector = graph.get_tensor_by_name('G_mapping_1/_Run/G_mapping/dlatents_out:0')
    generated_image = graph.get_tensor_by_name('G_synthesis_1/_Run/concat/concat:0')

    # Generate an image from the original image
    image = sess.run(generated_image, feed_dict={latent_vector: image_batch})

    # Rescale the pixel values from [-1, 1] to [0, 255]
    image = ((image + 1) * 127.5).astype(np.uint8)

    # Save the generated image
    output_dir = 'path/to/output/directory'
    os.makedirs(output_dir, exist_ok=True)
    output_image_path = os.path.join(output_dir, 'generated_image.png')
    cv2.imwrite(output_image_path, image[0])

print('Image generation complete!')

Mounted at /content/drive


OSError: ignored